In [1]:
import os


import numpy as np
import torch
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from torch.utils.data import Dataset
import logging

from datasets import load_dataset

raw_datasets = load_dataset("glue", 'stsb')




In [2]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoConfig
#from roberta import RobertaForSequenceClassification


model_name = "FacebookAI/roberta-base"
config = AutoConfig.from_pretrained(model_name)

config.hidden_dropout_prob=0.1
config.attention_probs_dropout_prob=0.01
#config.num_labels=2
tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = DebertaV2ForQuestionAnswering.from_pretrained("/data2/nusrat/work/bert-finetuned-squad_2/checkpoint-47500")



/home/kowsher/miniconda3/envs/LD/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
from transformers import AutoTokenizer
from transformers import AutoTokenizer, DataCollatorWithPadding

tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
# col_to_delete = ['idx']
col_to_delete = ['sentence1','sentence2']

def preprocessing_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True,max_length=512)

tokenized_dataset = raw_datasets.map(preprocessing_function, batched=True)

# tokenized_test_dataset = test_dataset.map(preprocessing_function, batched=True, remove_columns=col_to_delete)
# llama_tokenized_datasets = llama_tokenized_datasets.rename_column("target", "label")
# tokenized_train_dataset.set_format("torch")
tokenized_dataset.set_format("torch")

# Data collator for padding a batch of examples to the maximum length seen in the batch
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [4]:



import torch
import torch.nn as nn
from transformers import RobertaForSequenceClassification
from transformers.activations import ACT2FN

# Load the pre-trained model
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=1)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:

import propulsion

propulsion.PEFT(model, bias=True)


In [6]:



# Count of trainable parameters
total_trainable_params = 0
total =  0
# Print trainable parameters and count their total number
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"Parameter name: {name}, Shape: {param.shape}")
        
        total_trainable_params += param.numel()
    total+=param.numel()

print(f"Total trainable parameters:{total_trainable_params}, percentage:  {total_trainable_params/total}")

#677378

Parameter name: roberta.embeddings.word_embeddings.propulsion, Shape: torch.Size([768])
Parameter name: roberta.embeddings.position_embeddings.propulsion, Shape: torch.Size([768])
Parameter name: roberta.embeddings.token_type_embeddings.propulsion, Shape: torch.Size([768])
Parameter name: roberta.encoder.layer.0.attention.self.query.propulsion, Shape: torch.Size([768])
Parameter name: roberta.encoder.layer.0.attention.self.query.linear.bias, Shape: torch.Size([768])
Parameter name: roberta.encoder.layer.0.attention.self.key.propulsion, Shape: torch.Size([768])
Parameter name: roberta.encoder.layer.0.attention.self.key.linear.bias, Shape: torch.Size([768])
Parameter name: roberta.encoder.layer.0.attention.self.value.propulsion, Shape: torch.Size([768])
Parameter name: roberta.encoder.layer.0.attention.self.value.linear.bias, Shape: torch.Size([768])
Parameter name: roberta.encoder.layer.0.attention.output.dense.propulsion, Shape: torch.Size([768])
Parameter name: roberta.encoder.layer.0

In [7]:
import evaluate
import numpy as np
from sklearn import metrics
import torch
import numpy as np

import numpy as np
from scipy.stats import pearsonr
from scipy.stats import spearmanr
def compute_metrics(pred):
    preds = np.squeeze(pred.predictions)
    return {"MSE": ((preds - pred.label_ids) ** 2).mean().item(),
            "RMSE": (np.sqrt (( (preds - pred.label_ids) ** 2).mean())).item(),
            "MAE": (np.abs(preds - pred.label_ids)).mean().item(),
            "Pearson" : pearsonr(preds,pred.label_ids)[0],
            "Spearman's Rank":spearmanr(preds,pred.label_ids)[0]
           }

In [8]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='qnli_dir',
    learning_rate=2e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    weight_decay=0.0,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_total_limit=2,
    save_steps=100,
    logging_steps=100,
   
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],

    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()


/home/kowsher/miniconda3/envs/LD/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss,Mse,Rmse,Mae,Pearson,Spearman's rank
100,5.535000,1.705348,1.705348,1.305890,1.103649,0.755721,0.752246
200,1.215800,0.870145,0.870145,0.932816,0.746109,0.802964,0.823625
300,0.895300,0.752091,0.752091,0.867232,0.690335,0.838823,0.853420
400,0.780200,0.763455,0.763455,0.873759,0.696395,0.859154,0.866848
500,0.655600,0.609679,0.609679,0.780819,0.606413,0.861946,0.867801
600,0.634400,0.610143,0.610143,0.781117,0.595371,0.870042,0.880330
700,0.618400,0.579780,0.579780,0.761433,0.579045,0.876221,0.879199
800,0.575900,0.532471,0.532471,0.729706,0.557920,0.885803,0.885688
900,0.521000,0.537121,0.537121,0.732886,0.548254,0.881429,0.884857
1000,0.572100,0.585056,0.585056,0.764889,0.574847,0.879181,0.881768


TrainOutput(global_step=7200, training_loss=0.45801450279023914, metrics={'train_runtime': 398.4188, 'train_samples_per_second': 288.591, 'train_steps_per_second': 18.071, 'total_flos': 3747132291437424.0, 'train_loss': 0.45801450279023914, 'epoch': 20.0})